In [1]:
import numpy as np
import sys
import os
import neuro
import risp
import random
import time

import json

from encode import Chromagram
from encode import Encoder
from data_loader import DataLoader
import risp
import neuro
import eons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score


1.) Innitialize the neuroprocessor

In [2]:
risp_config = {
  "leak_mode": "all",
  "min_weight": -1,
  "max_weight": 1,
  "min_threshold": -1,
  "max_threshold": 1,
  "max_delay": 5,
  "discrete": False
}

proc = risp.Processor(risp_config) # RISP processor

net = neuro.Network() # Neuro network
net.set_properties(proc.get_network_properties()) # Set network properties


2.) Load the data

In [3]:
# First, split the data into training+validation and testing sets
X_temp, X_test, y_temp, y_test = train_test_split(DataLoader().chroma_files, DataLoader().numerical_label_roots, test_size=0.2, random_state=42)

# Now, split the training+validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# One-hot encode the labels
OneHot = OneHotEncoder(sparse_output=False)
y_train_encoded = OneHot.fit_transform(np.array(y_train).reshape(-1, 1))
y_val_encoded = OneHot.transform(np.array(y_val).reshape(-1, 1))
y_test_encoded = OneHot.transform(np.array(y_test).reshape(-1, 1))

3.) create template for eons

In [4]:
n_inputs = 12
n_hidden = 360
n_outputs = 12
n_neurons = n_inputs + n_hidden + n_outputs
n_synapses = 500


moa = neuro.MOA()
moa.seed(42)

In [5]:
def create_neuron(neuron_id, net, moa):
    neuron = net.add_node(neuron_id)
    net.randomize_node_properties(moa, neuron)
    return neuron

In [6]:
for i in range(n_inputs):
    neuron = create_neuron(i, net, moa)
    neuron.set("Threshold",0.75)
    net.add_input(i)
    
for i in range(n_outputs):
    neuron = create_neuron(i+n_inputs, net, moa)
    neuron.set("Threshold",0.75)
    net.add_output(i)
    
for i in range(n_hidden):
    neuron = create_neuron(i+n_inputs+n_outputs, net, moa)

In [7]:
print("Neuron 0's input ID:", net.get_node(0).input_id)
print("Neuron 1's input ID:", net.get_node(1).input_id)

Neuron 0's input ID: 0
Neuron 1's input ID: 1


In [8]:
for i in range(n_synapses):
    source = random.randint(0,n_neurons-1)
    dest = random.randint(0,n_neurons-1)
    synapse = net.add_or_get_edge(source, dest)
    net.randomize_edge_properties(moa, synapse)

4.) set up the eons

In [5]:
eo_params =  {
    "starting_nodes": 100,
    "starting_edges": 41,
    "merge_rate": 0,
    "population_size": 75,
    "multi_edges": 0,
    "crossover_rate": 0.5,
    "mutation_rate": 0.9,
    "selection_type": "tournament",
    "tournament_size_factor": 0.33,
    "tournament_best_net_factor": 0.9,
    "random_factor": 0,
    "num_mutations": 8,
    "node_mutations": { "Threshold": 1.0 },
    "net_mutations": { },
    "edge_mutations": { "Weight": 0.8, "Delay": 0.3 },
    "num_best": 2,

    "add_node_rate": 0.5,
    "delete_node_rate": 0.25,
    "add_edge_rate": 0.50,
    "delete_edge_rate": 0.25,
    "node_params_rate": 2.5,
    "edge_params_rate": 2.5,
    "net_params_rate" : 0
}


In [6]:
def read_network(fn):
    with open(fn, 'r') as f:
        s = f.read()
        j = json.loads(s)
        net = neuro.Network()
        net.from_json(j)
    return net

In [7]:
network3 = read_network('model_31.json')

In [8]:
print(network3.num_edges())
print(network3.num_nodes())

41
60


In [9]:
evolver = eons.EONS(eo_params)
evolver.set_template_network(network3)
pop = evolver.generate_population(eo_params,42)

In [10]:
def get_prediction(proc, x):
    # Load the chroma data from the npy file
    encoder=Encoder(x)
    
    proc.clear_activity()
    proc.apply_spikes(encoder.spikes)
    proc.run(encoder.time_steps * encoder.num_frames * 4) # you might adjust this duration based on your needs
    
    # Decoding the output to get the predicted label. You might need to adjust this
    predicted_index = proc.output_count_max(n_outputs)[0]
    # Convert index to one-hot encoded format
    one_hot_prediction = np.zeros(n_outputs)
    one_hot_prediction[predicted_index] = 1.0
    return one_hot_prediction 


In [11]:
def fitness(proc, net, X, y):
    proc.load_network(net)
    
    
    # Set up output tracking
    for i in range(n_outputs):
        proc.track_neuron_events(i)
    
    y_predict = [get_prediction(proc, x) for x in X]
    return accuracy_score(y_predict, y)


In [12]:
max_generations = 100
vals = []

for generation in range(max_generations):
    start_time = time.time()

    # Evaluate fitness of all networks in the population
    fitnesses = [fitness(proc, net.network, X_train, y_train_encoded) for net in pop.networks]
    
    # Track and print best fitness in the current generation
    best_fitness = max(fitnesses)
    vals.append(best_fitness)
    best_net = pop.networks[fitnesses.index(best_fitness)].network

    end_time = time.time()
    #elapsed time in format hh:mm:ss
    elapsed_time = time.strftime("%H:%M:%S", time.gmtime(end_time - start_time))

    print(f"Gen: {generation + 1}, Best Fitness: {best_fitness:.6f}, Max Nodes: {best_net.num_nodes()}, Elapsed Time: {elapsed_time}" )
    
    # Produce the next generation based on the current population's fitness``
    pop = evolver.do_epoch(pop, fitnesses, eo_params)

# Optionally, you can evaluate and print the performance of the best network on the test set here.
print("Evolution finished!")
print("Best fitness:", best_net)

Gen: 1, Best Fitness: 0.115297, Max Nodes: 112, Elapsed Time: 00:05:32
Gen: 2, Best Fitness: 0.145548, Max Nodes: 113, Elapsed Time: 00:05:34
Gen: 3, Best Fitness: 0.155822, Max Nodes: 112, Elapsed Time: 00:05:32
Gen: 4, Best Fitness: 0.163813, Max Nodes: 111, Elapsed Time: 00:05:31
Gen: 5, Best Fitness: 0.163813, Max Nodes: 115, Elapsed Time: 00:05:32
Gen: 6, Best Fitness: 0.163813, Max Nodes: 113, Elapsed Time: 00:05:33
Gen: 7, Best Fitness: 0.163813, Max Nodes: 113, Elapsed Time: 00:05:35


KeyboardInterrupt: 

In [13]:
best_net = pop.networks[fitnesses.index(best_fitness)].network
train = fitness(proc, best_net, X_train, y_train_encoded)
val = fitness(proc, best_net, X_val, y_val_encoded)

NameError: name 'fitnesses' is not defined

In [33]:
train

0.3030821917808219

In [48]:
print(best_net)

{
  "Associated_Data": {},
  "Edges": [
    {
      "from": 38,
      "to": 85,
      "values": [
        0.7164625254730839,
        4.0
      ]
    },
    {
      "from": 40,
      "to": 3,
      "values": [
        -0.3233169790644471,
        3.0
      ]
    },
    {
      "from": 95,
      "to": 182,
      "values": [
        0.17011236286957576,
        3.0
      ]
    },
    {
      "from": 40,
      "to": 98,
      "values": [
        -0.21349202636943476,
        3.0
      ]
    },
    {
      "from": 189,
      "to": 66,
      "values": [
        0.8448536502767479,
        1.0
      ]
    },
    {
      "from": 177,
      "to": 319,
      "values": [
        -0.7896587657252463,
        2.0
      ]
    },
    {
      "from": 131,
      "to": 116,
      "values": [
        -0.4840055810017524,
        3.0
      ]
    },
    {
      "from": 71,
      "to": 205,
      "values": [
        -0.4072769771812663,
        1.0
      ]
    },
    {
      "from": 185,
      "to": 70,
  

In [21]:
print(best_net.num_edges())
print(best_net.num_nodes())

50
48


In [ ]:
type(best_net)

neuro.Network

In [ ]:
def read_network(fn):
    with open(fn, 'r') as f:
        s = f.read()
        j = json.loads(s)
        net = neuro.Network()
        net.from_json(j)
    return net

In [62]:
network2 = read_network('model.json')

In [ ]:
type(network2)

neuro.Network

In [41]:
get_prediction(proc, X_test[0])

RuntimeError: risp::Processor::get_risp_network() network_id 0 does not exist